In [ ]:
"""

YZV 202E - Optimization for Data Science

Term Project

Efficient Post-Earthquake Relief Supply Distribution in Turkey: 
A Comparative Analysis of Particle Swarm Optimization and Mixed-Integer Linear Programming

Oğuz Kağan Pürçek
150220759

"""

In [8]:
#Particle Swarm Optimization

import pandas as pd
import random
import ipywidgets as widgets
from IPython.display import display

# Function that reads a csv file and returns a pandas dataframe 
def read_csv(file_name):
    try:
        data = pd.read_csv(file_name, sep=',', quotechar='"', engine='python') 
        print(f"{file_name} read successfully. The first 5 rows are as follows:\n") # Print first 5 rows
        print(data.head())
    except FileNotFoundError:
        print(f"{file_name} not found.") # Print error message
        return None
    except pd.errors.ParserError:
        print(f"An error occurred while reading {file_name} file.") # Print error message that parser could not read the file
        return None

    # Check if there are any missing values
    if data.isnull().values.any(): 
        print(f"{file_name} has missing values.")
    return data

# Function to calculate the cost of a route
def calculate_route_cost(source, destination, material, costs):
    key = (source, destination)

    # Check if the key exists in the dictionary
    if key in costs: 
        return costs[key][material]['cost'] 
    else:
        return float('inf') # Return infinity if the key does not exist

# Function to initialize particles for particle swarm optimization
def initialize_particles(num_particles, num_dimensions, demand):
    particles = []

    # For each particle, create a random particle
    for _ in range(num_particles): 
        particle = [[random.uniform(0, 1) for _ in range(num_dimensions)] for _ in range(len(demand))] # Create a random particle
        particles.append(particle) # Add the particle to the list of particles
    return particles

# Function to update the velocity of a particle
def update_particle_velocity(particle, best_particle, global_best_particle, inertia_weight, cognitive_weight, social_weight):
    new_velocity = []

    # For each dimension of the particle, calculate the new velocity
    for i in range(len(particle)): 
        particle_velocity = [] 
        for j in range(len(particle[i])):
            inertia_term = inertia_weight * particle[i][j] # Calculate the inertia term
            cognitive_term = cognitive_weight * random.uniform(0, 1) * (best_particle[i][j] - particle[i][j]) # Calculate the cognitive term
            social_term = social_weight * random.uniform(0, 1) * (global_best_particle[i][j] - particle[i][j]) # Calculate the social term 
            particle_velocity.append(inertia_term + cognitive_term + social_term) # Calculate the new velocity 
        new_velocity.append(particle_velocity) # Add the new velocity to the list of velocities
    return new_velocity

# Function to update the position of a particle
def update_particle_position(particle, velocity):
    new_position = []

    # For each dimension of the particle, calculate the new position
    for i in range(len(particle)): 
        particle_position = []
        for j in range(len(particle[i])):
            particle_position.append(particle[i][j] + velocity[i][j]) 
        new_position.append(particle_position)
    return new_position

# Function to evaluate a particle's fitness and generate routes
def evaluate_particle(particle, supply, demand, costs):
    total_cost = 0
    routes = []

    # For each demand, find the best source for each material
    for i in range(len(demand)):
        destination = demand[i]['city'] # Get the destination city
        best_source_food = None 
        best_source_water = None
        best_source_medicine = None
        min_cost_food = float('inf')
        min_cost_water = float('inf')
        min_cost_medicine = float('inf')
    
        for j in range(len(supply)):
            source = supply[j]['center'] # Get the source center
            quantity = particle[i][j] # Get the quantity of the material

            # Calculate the cost of the transportation of each material
            cost_food = calculate_route_cost(source, destination, 'food', costs) 
            cost_water = calculate_route_cost(source, destination, 'water', costs) 
            cost_medicine = calculate_route_cost(source, destination, 'medicine', costs) 

            # Check if the cost of the transportation of each material is less than the minimum cost 
            if cost_food * quantity < min_cost_food:
                best_source_food = source # If it is, update the best source of food
                min_cost_food = cost_food * quantity # If it is, update the minimum cost

            if cost_water * quantity < min_cost_water:
                best_source_water = source # Update the best source of water
                min_cost_water = cost_water * quantity # Update the minimum cost of water

            if cost_medicine * quantity < min_cost_medicine:
                best_source_medicine = source # Update the best source of medicine
                min_cost_medicine = cost_medicine * quantity # Update the minimum cost of medicine

        # Add the minimum cost of each material to the total cost
        total_cost += min(min_cost_food, min_cost_water, min_cost_medicine)

        # Add the best source of each material to the list of routes
        routes.append(f"Route({destination}):\n"
                      f"\t- Food: Source({best_source_food}), Cost({min_cost_food})\n"
                      f"\t- Water: Source({best_source_water}), Cost({min_cost_water})\n"
                      f"\t- Medicine: Source({best_source_medicine}), Cost({min_cost_medicine})")

    return total_cost, routes

# Function to perform Particle Swarm Optimization
def particle_swarm_optimization(num_particles, num_dimensions, max_iterations, supply, demand, costs):
    # Initialize the parameters
    inertia_weight = 0.7 
    cognitive_weight = 1.4 
    social_weight = 1.4 

    # Initialize the particles
    particles = initialize_particles(num_particles, num_dimensions, demand)
    best_particle = particles[0]
    best_fitness = float('inf')
    global_best_particle = best_particle 

    # For each iteration, update the velocity and position of the particle
    for _ in range(max_iterations):
        for particle in particles:
            fitness, _ = evaluate_particle(particle, supply, demand, costs) # Evaluate the fitness of the particle

            # Check if the fitness of the particle is less than the best fitness
            if fitness < best_fitness:
                best_fitness = fitness
                best_particle = particle
            
            # Update the global best particle
            global_best_particle = update_global_best_particle(particles, global_best_particle, supply, demand, costs)

            # Update the velocity and position of the particle
            velocity = update_particle_velocity(particle, best_particle, global_best_particle, inertia_weight, cognitive_weight, social_weight)
            particle = update_particle_position(particle, velocity)

    return global_best_particle

# Function to update the global best particle in PSO
def update_global_best_particle(particles, global_best_particle, supply, demand, costs):
    global_best_fitness, _ = evaluate_particle(global_best_particle, supply, demand, costs) # Evaluate the fitness of the global best particle

    # For each particle, evaluate the fitness of the particle
    for particle in particles:
        fitness, _ = evaluate_particle(particle, supply, demand, costs)

        # Check if the fitness of the particle is less than the global best fitness
        if fitness < global_best_fitness:
            global_best_fitness = fitness
            global_best_particle = particle

    return global_best_particle

# Main code

# Read supply, demand, and costs data from CSV files
supply_data = read_csv('distributioncenter.csv')
demand_data = read_csv('effectedcities.csv')
costs_data = read_csv('transportinfo.csv')

# Check if the data was read successfully
if any(data is None for data in [supply_data, demand_data, costs_data]):
    print("All files could not be read successfully. Please check your files.")
else:
    supply = []
    demand = []
    costs = {}

    # Process supply data
    for _, row in supply_data.iterrows():
        center = row['Center']
        food = int(row['SupplyFood'].replace(',', ''))
        water = int(row['SupplyWater'].replace(',', ''))
        medicine = int(row['SupplyMedicine'].replace(',', ''))
        supply.append({
            'center': center,
            'food': food,
            'water': water,
            'medicine': medicine
        })

    # Process demand data
    for _, row in demand_data.iterrows():
        city = row['City']
        food = int(row['DemandFood'].replace(',', ''))
        water = int(row['DemandWater'].replace(',', ''))
        medicine = int(row['DemandMedicine'].replace(',', ''))
        demand.append({
            'city': city,
            'food': food,
            'water': water,
            'medicine': medicine
        })

    # Process costs data
    for _, row in costs_data.iterrows():
        source = row['Source']
        destination = row['Destination']
        distance = row['Distance']
        road_cost = row['Road Cost']
        air_cost = row['Air Cost']

        if (source, destination) not in costs:
            costs[(source, destination)] = {}

        costs[(source, destination)] = {
            'food': {'cost': road_cost, 'distance': distance, 'road': 'Open', 'air': 'Open', 'air_cost': air_cost},
            'water': {'cost': road_cost, 'distance': distance, 'road': 'Open', 'air': 'Open', 'air_cost': air_cost},
            'medicine': {'cost': road_cost, 'distance': distance, 'road': 'Open', 'air': 'Open', 'air_cost': air_cost}
        }

    num_particles = 50
    num_dimensions = len(supply)
    max_iterations = 100

    def solve_button_clicked(button):
        # Solve the PSO problem
        best_particle = particle_swarm_optimization(num_particles, num_dimensions, max_iterations, supply, demand, costs)

        # Print the best particle and its fitness
        print("Best Particle:")
        print(best_particle)
        fitness, routes = evaluate_particle(best_particle, supply, demand, costs)
        print("Fitness:")
        print(fitness)
        print("Routes and Costs:")
        for route in routes:
            print(route)

    # Supply sliders
    supply_sliders = {}
    for item in supply:
        center = item['center']
        supply_sliders[center] = {
            'food': widgets.IntSlider(value=item['food'], min=0, max=1000000, description='Food'),
            'water': widgets.IntSlider(value=item['water'], min=0, max=1000000, description='Water'),
            'medicine': widgets.IntSlider(value=item['medicine'], min=0, max=1000000, description='Medicine')
        }

    # Demand sliders
    demand_sliders = {}
    for item in demand:
        city = item['city']
        demand_sliders[city] = {
            'food': widgets.IntSlider(value=item['food'], min=0, max=1000000, description='Food'),
            'water': widgets.IntSlider(value=item['water'], min=0, max=1000000, description='Water'),
            'medicine': widgets.IntSlider(value=item['medicine'], min=0, max=1000000, description='Medicine')
        }
        
    # Display the sliders
    for center in supply_sliders:
        print(f"Supply Center: {center}")
        for item in supply_sliders[center]:
            display(supply_sliders[center][item])
        print()

    for city in demand_sliders:
        print(f"Demand City: {city}")
        for item in demand_sliders[city]:
            display(demand_sliders[city][item])
        print()

    # Initialize supply sliders
    for center in supply_sliders:
        for item in supply_sliders[center]:
            supply_sliders[center][item].value = supply_sliders[center][item].value

    # Initialize demand sliders
    for city in demand_sliders:
        for item in demand_sliders[city]:
            demand_sliders[city][item].value = demand_sliders[city][item].value

    # Solve button
    solve_button = widgets.Button(description='Solve')
    solve_button.on_click(solve_button_clicked)

    # Display the solve button
    display(solve_button)

distributioncenter.csv read successfully. The first 5 rows are as follows:

     Center  Population SupplyFood SupplyWater SupplyMedicine
0  Istanbul  15,907,951  3,981,988   3,185,590      2,389,193
1    Ankara   5,782,285  1,445,571   1,156,457        867,343
2    Samsun   1,368,488    342,122     273,698        205,273
3   Antalya   2,688,004    672,001     537,601        403,201
4     İzmir   4,462,056  1,115,514     892,411        669,308
effectedcities.csv read successfully. The first 5 rows are as follows:

         City Population DamageLevel DemandFood DemandWater DemandMedicine
0       Adana  2,274,106        High    547,023     436,819        328,114
1    Adıyaman    635,169    Moderate    152,441     121,952         91,463
2  Diyarbakır  1,804,880        High    433,174     346,539        259,904
3   Gaziantep  2,154,051    Moderate    516,972     413,578        310,183
4       Hatay  1,686,043         Low    337,209     269,767        202,325
transportinfo.csv read success

IntSlider(value=1000000, description='Food', max=1000000)

IntSlider(value=1000000, description='Water', max=1000000)

IntSlider(value=1000000, description='Medicine', max=1000000)


Supply Center: Ankara


...

IntSlider(value=142953, description='Food', max=1000000)

IntSlider(value=114362, description='Water', max=1000000)

IntSlider(value=85771, description='Medicine', max=1000000)

Button(description='Solve', style=ButtonStyle())

Best Particle:
[[0.5414530393299944, 0.006365693267111228, 0.04952427056466435, 0.16165137495836446, 0.0027185165826469904], [0.10577568214505906, 0.21403499337705134, 0.9603919726371569, 0.20763925604239075, 0.22050961273437208], [0.6249005055205488, 0.05183514878448925, 0.06980160958266202, 0.9229977114118035, 0.7528000912144082], [0.5617420771538737, 0.012623214335116817, 0.6405275666522521, 0.9194690523710389, 0.10220863135994962], [0.4971930054185343, 0.1759368079200465, 0.19127605988716223, 0.019054830590247707, 0.6890415565120016], [0.5936064330560782, 0.20738163303539692, 0.6526213441385958, 0.7709884252655171, 0.05643883550425133], [0.6710990912352032, 0.7644763574813306, 0.22115119315212117, 0.5602531463213157, 0.4163107974512069], [0.910538815473769, 0.4986880671455577, 0.1333301741178543, 0.22426898666305084, 0.10522599388394549], [0.1888825600132069, 0.9290149773360935, 0.492639840119941, 0.7751227278355691, 0.8771345550241532], [0.46432734778971674, 0.8470572215708432, 0.

In [9]:
#Mixed-Integer Linear Programming

import pandas as pd
from pulp import *
import ipywidgets as widgets
from IPython.display import display

# Define a function to read the data from CSV files
def read_csv(file_name):
    try:
        # Read the data, assuming that the values are separated by a comma
        data = pd.read_csv(file_name, sep=None, thousands=',', quotechar='"', engine='python')
        print(f"{file_name} read successfully. The first 5 rows are as follows:\n")
        print(data.head())
        
    except FileNotFoundError:
        # Handle the case where the file doesn't exist
        print(f"{file_name} not found.")
        return None
    
    except pd.errors.ParserError:
        # Handle other potential errors
        print(f"An error occurred while reading {file_name} file.")
        return None
    
    # Check for missing values in the data
    if data.isnull().values.any():
        print(f"{file_name} has missing values.")
    return data

# Read the data files
transport_info = read_csv('transportinfo.csv')
petrol_info = read_csv('petrolinformation.csv')
distribution_centers = read_csv('distributioncenter.csv')
affected_cities = read_csv('effectedcities.csv')

# Check if any of the dataframes is None before proceeding
if any(df is None for df in [transport_info, petrol_info, distribution_centers, affected_cities]):
    print("All files could not be read successfully. Please check your files.")
else:
    # Create the 'prob' variable to contain the problem data
    prob = LpProblem("Earthquake_Relief_Supply_Distribution", LpMinimize)
    
    # Build dictionaries from dataframes to represent supply, demand, petrol supply, consumption rate and costs
    # These dictionaries will be used to build constraints for the linear programming problem
    supply = {row['Center']: {'food': row['SupplyFood'],
                              'water': row['SupplyWater'],
                              'medicine': row['SupplyMedicine']} 
              for _, row in distribution_centers.iterrows()}

    demand = {row['City']: {'food': row['DemandFood'],
                            'water': row['DemandWater'],
                            'medicine': row['DemandMedicine']} 
              for _, row in affected_cities.iterrows()}

    petrol_supply = {row['Center']: row['CurrentPetrol(L)'] 
                     for _, row in petrol_info.iterrows()}

    consumption_rate = {row['Center']: row['ConsumptionRate(L/KM)'] 
                        for _, row in petrol_info.iterrows()}

    costs = {(row['Source'], row['Destination']): {'cost': row['Road Cost'],
                                                   'distance': row['Distance'],
                                                   'road': row['Transport Road'],
                                                   'air': row['Transport Air'],
                                                   'air_cost': row['Air Cost']}
             for _, row in transport_info.iterrows()}

    # Create the decision variables
    vars = LpVariable.dicts("Route", [(s, d, g) for s in supply for d in demand for g in ['food', 'water', 'medicine']],
                            lowBound=0, cat='Integer')

    # Define the objective function, which is to minimize the total transport cost
    prob += lpSum([costs[(s, d)]['cost'] * vars[(s, d, g)] +
                   costs[(s, d)]['air_cost'] * vars[(s, d, g)]
                   for s in supply for d in demand for g in ['food', 'water', 'medicine']]), "Total Cost of Transport"

    # Add constraints to the problem
    # The total supply from a center can't exceed its current supply, and the total demand for a city has to be satisfied.
    for s in supply:
        for g in ['food', 'water', 'medicine']:
            prob += lpSum(vars[(s, d, g)] for d in demand) <= supply[s][g], f"Sum of {g} from Center {s}"

    for d in demand:
        for g in ['food', 'water', 'medicine']:
            prob += lpSum(vars[(s, d, g)] for s in supply) >= demand[d][g], f"Sum of {g} to City {d}"
    
    # The supply from a center to a city can't exceed the center's supply,
    # and the total supply to a city has to equal the city's demand.
    for p in petrol_supply:
        for d in demand:
            for g in ['food', 'water', 'medicine']:
                prob += vars[(p, d, g)] <= supply[p][g], f"Supply limit of {g} from Center {p} to City {d}"

    for d in demand:
        for g in ['food', 'water', 'medicine']:
            prob += lpSum(vars[(s, d, g)] for s in supply) == demand[d][g], f"Demand satisfaction of {g} in City {d}"
    
    # Define a button click event handler to solve the LP problem
    def solve_button_clicked(button):
        # Solve the LP problem
        prob.solve()

        # Print the status of the solved LP
        print("Status:", LpStatus[prob.status])

        # Print the total cost of transport
        print("Total Cost of Transport =", value(prob.objective))

        # Print the optimal solution
        for v in prob.variables():
            if v.varValue is not None and v.varValue > 0:
                print(v.name, "=", v.varValue)

    # Create sliders for supply and demand, allowing user interaction to adjust values
    supply_sliders = {center: {item: widgets.IntSlider(value=supply[center][item], min=0, max=1000000, description=item.capitalize()) 
                               for item in supply[center]} 
                      for center in supply}

    demand_sliders = {city: {item: widgets.IntSlider(value=demand[city][item], min=0, max=1000000, description=item.capitalize()) 
                             for item in demand[city]} 
                      for city in demand}

    # Display the sliders
    for center in supply_sliders:
        print(f"Supply Center: {center}")
        for item in supply_sliders[center]:
            display(supply_sliders[center][item])
        print()

    for city in demand_sliders:
        print(f"Demand City: {city}")
        for item in demand_sliders[city]:
            display(demand_sliders[city][item])
        print()

    # Solve button
    solve_button = widgets.Button(description='Solve')
    solve_button.on_click(solve_button_clicked)

    # Display the solve button
    display(solve_button)

transportinfo.csv read successfully. The first 5 rows are as follows:

     Source Destination  Distance Transport Road  Road Cost Transport Air  \
0  Istanbul       Adana       892           Open          1          Open   
1  Istanbul    Adıyaman       939           Open          1          Open   
2  Istanbul  Diyarbakır      1273         Closed          1          Open   
3  Istanbul   Gaziantep      1220           Open          1          Open   
4  Istanbul       Hatay      1210         Closed          1          Open   

   Air Cost  
0         3  
1         3  
2         3  
3         3  
4         3  
petrolinformation.csv read successfully. The first 5 rows are as follows:

     Center  CurrentPetrol(L)  ConsumptionRate(L/KM)
0  Istanbul           5000000                      1
1    Ankara           2000000                      1
2    Samsun           1000000                      1
3   Antalya           1500000                      1
4     İzmir           2500000             

IntSlider(value=1000000, description='Food', max=1000000)

IntSlider(value=1000000, description='Water', max=1000000)

IntSlider(value=1000000, description='Medicine', max=1000000)


Supply Center: Ankara


...

IntSlider(value=142953, description='Food', max=1000000)

IntSlider(value=114362, description='Water', max=1000000)

IntSlider(value=85771, description='Medicine', max=1000000)

Button(description='Solve', style=ButtonStyle())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/jr/hc_sx4xx7xs0hgcg4p10r67w0000gn/T/5102ba99064c4a89966d5b75758d1b14-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/jr/hc_sx4xx7xs0hgcg4p10r67w0000gn/T/5102ba99064c4a89966d5b75758d1b14-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 251 COLUMNS
At line 1407 RHS
At line 1654 BOUNDS
At line 1820 ENDATA
Problem MODEL has 246 rows, 165 columns and 660 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.09937e+07 - 0.00 seconds
Cgl0004I processed model has 45 rows, 165 columns (165 integer (0 of which binary)) and 297 elements
Cutoff increment increased from 1e-05 to 3.9999
Cbc0012I Integer solution of 30993700 found by DiveCoefficient after 0 itera

In [1]:
# Map

import folium

# Create a map centered on Turkey
m = folium.Map(location=[39.9255, 32.8662], zoom_start=6)

# Define the coordinates of Istanbul, Antalya, Izmir, Samsun, and Ankara
istanbul_coords = (41.00527, 28.97696)
antalya_coords = (36.88414, 30.70563)
izmir_coords = (38.41885, 27.12872)
samsun_coords = (41.292782, 36.33128)
ankara_coords = (39.92077, 32.85411)

# Mark Istanbul, Antalya, Izmir, Samsun, and Ankara with red markers
folium.Marker(
    location=istanbul_coords,
    icon=folium.Icon(color='red', icon='info-sign'),
    tooltip="Istanbul"
).add_to(m)

folium.Marker(
    location=antalya_coords,
    icon=folium.Icon(color='red', icon='info-sign'),
    tooltip="Antalya"
).add_to(m)

folium.Marker(
    location=izmir_coords,
    icon=folium.Icon(color='red', icon='info-sign'),
    tooltip="Izmir"
).add_to(m)

folium.Marker(
    location=samsun_coords,
    icon=folium.Icon(color='red', icon='info-sign'),
    tooltip="Samsun"
).add_to(m)

folium.Marker(
    location=ankara_coords,
    icon=folium.Icon(color='red', icon='info-sign'),
    tooltip="Ankara"
).add_to(m)

# Define the coordinates of Kahramanmaraş, Hatay, Adıyaman, Osmaniye, Gaziantep, Adana, Kilis,
# Malatya, Diyarbakır, Şanlıurfa, and Elazığ
kahramanmaras_coords = (37.585831, 36.937149)
hatay_coords = (36.401849, 36.34981)
adiyaman_coords = (37.764751, 38.278561)
osmaniye_coords = (37.213026, 36.176261)
gaziantep_coords = (37.06622, 37.38332)
adana_coords = (37.00000, 35.321333)
kilis_coords = (36.718399, 37.12122)
malatya_coords = (38.35519, 38.30946)
diyarbakir_coords = (37.91441, 40.230629)
sanliurfa_coords = (37.159149, 38.796909)
elazig_coords = (38.680969, 39.226398)

# Mark Kahramanmaraş, Hatay, Adıyaman, Osmaniye, Gaziantep, Adana, Kilis,
# Malatya, Diyarbakır, Şanlıurfa, and Elazığ with yellow markers
folium.Marker(
    location=kahramanmaras_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Kahramanmaras"
).add_to(m)

folium.Marker(
    location=hatay_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Hatay"
).add_to(m)

folium.Marker(
    location=adiyaman_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Adıyaman"
).add_to(m)

folium.Marker(
    location=osmaniye_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Osmaniye"
).add_to(m)

folium.Marker(
    location=gaziantep_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Gaziantep"
).add_to(m)

folium.Marker(
    location=adana_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Adana"
).add_to(m)

folium.Marker(
    location=kilis_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Kilis"
).add_to(m)

folium.Marker(
    location=malatya_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Malatya"
).add_to(m)

folium.Marker(
    location=diyarbakir_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Diyarbakır"
).add_to(m)

folium.Marker(
    location=sanliurfa_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Şanlıurfa"
).add_to(m)

folium.Marker(
    location=elazig_coords,
    icon=folium.Icon(color='yellow', icon='info-sign'),
    tooltip="Elazığ"
).add_to(m)

# Define supply routes between cities, each with a specific type of supply (e.g. 'food', 'medicine', 'water') 
# and its corresponding color
routes = {
    ('Ankara', 'Adana', 'medicine'): 'green',
    ('Ankara', 'Adana', 'water'): 'blue',
    ('Ankara', 'Adıyaman', 'food'): 'yellow',
    ('Ankara', 'Diyarbakır', 'food'): 'yellow',
    ('Ankara', 'Gaziantep', 'water'): 'blue',
    ('Ankara', 'Hatay', 'food'): 'yellow',
    ('Ankara', 'Hatay', 'medicine'): 'green',
    ('Ankara', 'Kahramanmaras', 'water'): 'blue',
    ('Ankara', 'Kilis', 'water'): 'blue',
    ('Ankara', 'Şanlıurfa', 'food'): 'yellow',
    ('Antalya', 'Adana', 'food'): 'yellow',
    ('Antalya', 'Adıyaman', 'water'): 'blue',
    ('Antalya', 'Elazığ', 'food'): 'yellow',
    ('Antalya', 'Hatay', 'water'): 'blue',
    ('Antalya', 'Şanlıurfa', 'water'): 'blue',
    ('Istanbul', 'Adıyaman', 'medicine'): 'green',
    ('Istanbul', 'Diyarbakır', 'medicine'): 'green',
    ('Istanbul', 'Diyarbakır', 'water'): 'blue',
    ('Istanbul', 'Elazığ', 'food'): 'yellow',
    ('Istanbul', 'Kahramanmaras', 'food'): 'yellow',
    ('Istanbul', 'Kilis', 'food'): 'yellow',
    ('Istanbul', 'Malatya', 'water'): 'blue',
    ('Istanbul', 'Osmaniye', 'food'): 'yellow',
    ('Istanbul', 'Osmaniye', 'medicine'): 'green',
    ('Istanbul', 'Şanlıurfa', 'medicine'): 'green',
    ('Samsun', 'Elazığ', 'water'): 'blue',
    ('Samsun', 'Hatay', 'water'): 'blue',
    ('Samsun', 'Kilis', 'medicine'): 'green',
    ('Samsun', 'Malatya', 'food'): 'yellow',
    ('Samsun', 'Malatya', 'medicine'): 'green',
    ('Izmir', 'Adıyaman', 'water'): 'blue',
    ('Izmir', 'Elazığ', 'medicine'): 'green',
    ('Izmir', 'Gaziantep', 'food'): 'yellow',
    ('Izmir', 'Gaziantep', 'medicine'): 'green',
    ('Izmir', 'Kahramanmaras', 'medicine'): 'green',
    ('Izmir', 'Osmaniye', 'water'): 'blue'
}

# Draw these supply routes on the map using lines (PolyLine), with the color indicating the type of supply
for route, color in routes.items():
    start_coords = eval(route[0].lower().replace('ı', 'i').replace('ş', 's') + '_coords')
    end_coords = eval(route[1].lower().replace('ı', 'i').replace('ş', 's').replace('ğ', 'g') + '_coords')
    folium.PolyLine(
        locations=[start_coords, end_coords],
        color=color,
        weight=2
    ).add_to(m)

# Add legend to the map to explain what each color means
# This is done by adding a custom HTML element to the map
legend_html = '''
<div style="position: fixed; 
            bottom: 40px; left: 40px; width: 120px; height: 140px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            text-align: center;
            ">
    <strong>Colors</strong>
    <hr style="border:1px solid grey;">
    <div style="padding: -8px;">
        <span style="color:green;">MEDICINE</span>
    </div>
    <div style="padding: -8px;">
        <span style="color:blue;">WATER</span>
    </div>
    <div style="padding: -9px;">
        <span style="color:yellow;">FOOD</span>
    </div>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Display the final map
m

/var/folders/jr/hc_sx4xx7xs0hgcg4p10r67w0000gn/T/ipykernel_1686/4186614636.py:64: UserWarning: color argument of Icon should be one of: {'purple', 'pink', 'darkpurple', 'lightred', 'cadetblue', 'red', 'white', 'orange', 'blue', 'green', 'darkred', 'darkgreen', 'lightgray', 'darkblue', 'beige', 'gray', 'lightgreen', 'black', 'lightblue'}.
  icon=folium.Icon(color='yellow', icon='info-sign'),
/var/folders/jr/hc_sx4xx7xs0hgcg4p10r67w0000gn/T/ipykernel_1686/4186614636.py:70: UserWarning: color argument of Icon should be one of: {'purple', 'pink', 'darkpurple', 'lightred', 'cadetblue', 'red', 'white', 'orange', 'blue', 'green', 'darkred', 'darkgreen', 'lightgray', 'darkblue', 'beige', 'gray', 'lightgreen', 'black', 'lightblue'}.
  icon=folium.Icon(color='yellow', icon='info-sign'),
/var/folders/jr/hc_sx4xx7xs0hgcg4p10r67w0000gn/T/ipykernel_1686/4186614636.py:76: UserWarning: color argument of Icon should be one of: {'purple', 'pink', 'darkpurple', 'lightred', 'cadetblue', 'red', 'white', 'o